## 1. Upload raw data files into S3

In [ ]:
!pip install pyspark

In [ ]:
!pip install boto3

1.1 Imports and Configs 

In [17]:
import pandas as pd

import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, to_timestamp, to_date

In [26]:
config = configparser.ConfigParser()
config.read('configs/global.cfg')

KEY = config.get('AWS', 'AWS_ACCESS_KEY_ID')
SECRET = config.get('AWS','AWS_SECRET_ACCESS_KEY')

input_path = config.get('PATH', 'INPUT_DATA_FOLDER')
output_path = config.get('PATH', 'OUTPUT_DATA_FOLDER')

raw_flight_data_path = input_path + config.get('PATH', 'FLIGHTS_RAW_FOLDER')
raw_tweets_data_path = input_path + config.get('PATH', 'TWEETS_RAW_FOLDER')

1.2 Create Spark Session

In [20]:
def create_spark_session():
    """
    - Create or retrieve existing spark session
    
    Returns: 
        spark -- SparkSession object 
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [22]:
spark = create_spark_session()

sc = spark.sparkContext

1.3 Load flights df

In [66]:
flights_df = spark.read.options( 
            recursiveFileLookup=True , 
            inferSchema=True, 
            header=True)\
        .csv( raw_flight_data_path )

In [ ]:
from pyspark.sql.functions import countDistinct, desc

flights_df.select("callsign").groupBy("callsign")\
    .agg( countDistinct("callsign").alias("count") )\
    .sort( desc("count") )\
    .limit(10).toPandas()

In [ ]:
flights_df.limit(20).toPandas()

In [64]:
flights_staging = flights_df.selectExpr( "callsign", "icao24 as trasponder_id", 
                      "registration as aircraft_id", "typecode as aircraft_type",
                     "origin as depart_airport_id", "destination as arrival_airport_id",
                        "firstseen as depart_at", "lastseen as arrival_at")\
    .filter("arrival_airport_id is not null")

Cardinality of sample test 2021-03-12: 6.109.738

1.3.1 Enrich airport info

In [96]:
from pyspark import SparkFiles
spark.sparkContext.addFile("https://ourairports.com/data/airports.csv")

airports_df = spark.read.csv("file://" +SparkFiles.get("airports.csv"), header=True, inferSchema= True)


In [107]:
airports_staging = airports_df.selectExpr("id", "ident as code", "type", "name", "iso_country", "municipality")

63078

Cardinality full dataset test 2021-03-10: 63.078 rows

1.4 Load Tweets df

In [67]:
tweets_df = spark.read.options( 
            recursiveFileLookup=True , 
            inferSchema=True, 
            header=True)\
        .json( raw_tweets_data_path )

In [ ]:
tweets_df.dtypes

In [106]:
tweets_staging = tweets_df.select(['date', 'keywords', 'location.country', 'tweet_id'])\
    .filter( col("location").isNotNull() )

2302853

Cardinality of sample test 2021-03-12 : 2.302.853 rows

2 Load to output (S3)

In [110]:
tweets_staging.write.parquet(output_path + "tweets.parquet", mode="overwrite")

In [114]:
flights_staging.write.parquet(output_path + "flights.parquet", mode="overwrite")

In [113]:
airports_staging.write.parquet(output_path + "airports.parquet", mode="overwrite")

In [116]:
spark.read.parquet( output_path + "flights.parquet" ).count()

6109738

3 Stop spark

In [ ]:
spark.stop()